In [1]:
import quandl
import pandas as pd
import numpy as np
import datetime
from sklearn.neural_network import MLPRegressor
from sklearn import preprocessing, cross_validation, svm
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
%matplotlib inline

/home/aleespa/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = quandl.get("WIKI/MSFT")

In [3]:
func = ['mean','std','skew','kurt']
time = [2,7,30,60,120,252]
k = df.shape[0]-252
Z = pd.DataFrame()
for f in func:
    for t in time:
        aux = pd.DataFrame()
        for i in range(252):
            aux[df.index[i+k]] = list(df.iloc[i+k-t:i+k]['Adj. Close'])
        Z[f+str(t)] = aux.apply(f,axis=0)
Z.tail()

,mean2,mean7,mean30,mean60,mean120,mean252,std2,std7,std30,std60,...,skew30,skew60,skew120,skew252,kurt2,kurt7,kurt30,kurt60,kurt120,kurt252
2018-03-21,93.010,94.261429,92.642667,90.988500,85.923496,77.358864,0.169706,1.275832,2.497495,3.216538,...,-1.132983,-0.356039,-0.287449,0.412976,NaN,2.494047,2.105830,-0.960705,-0.816861,-1.133319
2018-03-22,92.805,93.648571,92.681000,91.104833,86.086737,77.472270,0.459619,0.818320,2.485449,3.139982,...,-1.193683,-0.402237,-0.289892,0.402984,NaN,-1.752289,2.291619,-0.857901,-0.822501,-1.151439
2018-03-23,91.135,92.988571,92.687000,91.176167,86.222670,77.577813,1.902117,1.595697,2.477986,3.058279,...,-1.200628,-0.401787,-0.294434,0.393050,NaN,2.808938,2.351568,-0.775017,-0.807490,-1.169875
2018-03-26,88.485,92.035714,92.763333,91.205833,86.336687,77.669797,1.845549,2.643211,2.254735,3.009543,...,-0.713123,-0.369305,-0.286647,0.382020,NaN,0.792698,0.508059,-0.798161,-0.793047,-1.182563
2018-03-27,90.480,91.978571,92.952667,91.340333,86.500563,77.788596,4.666905,2.592974,2.081467,2.939273,...,-0.779877,-0.412148,-0.288054,0.374272,NaN,0.932427,1.155710,-0.715684,-0.797423,-1.197794


In [4]:
Z.shape

(252, 24)

In [5]:
df[['Adj. Close']].shift(-1)
W = pd.merge(Z, df[['Adj. Close']].shift(-1),left_index=True,right_index=True)
W = W[:-1]

In [6]:
W.head()

,mean2,mean7,mean30,mean60,mean120,mean252,std2,std7,std30,std60,...,skew60,skew120,skew252,kurt2,kurt7,kurt30,kurt60,kurt120,kurt252,Adj. Close
2017-03-27,63.896764,63.763902,63.524659,62.687714,60.603842,56.280720,0.076550,0.279809,0.336093,1.060381,...,-0.351811,-0.486165,-0.093777,NaN,3.081831,0.403997,-1.382042,-0.971660,-1.155918,64.255984
2017-03-28,64.009943,63.828576,63.573350,62.728526,60.672736,56.331228,0.083509,0.292053,0.302928,1.065710,...,-0.409672,-0.504710,-0.104637,NaN,5.347478,-0.804916,-1.325835,-0.933185,-1.153629,64.433133
2017-03-29,64.162488,63.887626,63.604796,62.773923,60.744646,56.378025,0.132223,0.334126,0.323213,1.071880,...,-0.459038,-0.517070,-0.114165,NaN,4.049286,-0.568833,-1.248117,-0.900830,-1.154206,64.669332
2017-03-30,64.344558,63.963547,63.634321,62.834663,60.814793,56.424232,0.125263,0.393038,0.356527,1.060678,...,-0.469581,-0.532123,-0.122820,NaN,2.591308,-0.319863,-1.190701,-0.856444,-1.155077,64.816957
2017-03-31,64.551233,64.174439,63.673031,62.892167,60.886099,56.470691,0.167018,0.294412,0.402431,1.065125,...,-0.480649,-0.544022,-0.130517,NaN,-0.420144,0.137042,-1.065778,-0.808319,-1.155463,64.511866


In [7]:
X = W.drop(['Adj. Close'],1).fillna(0)
y = W[['Adj. Close']]

In [8]:
mm = MinMaxScaler()
mm.fit(X)
Xs = pd.DataFrame(mm.transform(X))

In [9]:
Xt, Xv, yt, yv = cross_validation.train_test_split(Xs, y, test_size = 0.3)
print Xt.shape

(175, 24)


In [10]:
model = MLPRegressor()

In [11]:
param_grid = dict(
activation = ['identity', 'logistic', 'tanh', 'relu'],
solver = ['lbfgs', 'sgd', 'adam'],
learning_rate = ['constant', 'invscaling', 'adaptive'],
alpha = np.arange(0.001,0.003,0.001)
)

In [12]:
grid = GridSearchCV(cv = 3, 
                    estimator = model,
                    n_jobs = -1,
                    param_grid=param_grid,
                    verbose = True
)

In [13]:
grid.fit(Xs,y)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


/home/aleespa/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/aleespa/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/aleespa/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/aleespa/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: M

GridSearchCV(cv=3, error_score='raise',
       estimator=MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'alpha': array([ 0.001,  0.002]), 'learning_rate': ['constant', 'invscaling', 'adaptive'], 'activation': ['identity', 'logistic', 'tanh', 'relu'], 'solver': ['lbfgs', 'sgd', 'adam']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [17]:
model = grid.best_estimator_
model.fit(Xt,yt)

MLPRegressor(activation='identity', alpha=0.002, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='invscaling',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [18]:
print model.score(Xv,yv)
print model.score(Xt,yt)

0.979597394577
0.985778720632


In [25]:
model.fit(Xs,y)

MLPRegressor(activation='identity', alpha=0.002, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='invscaling',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [ ]:
pred = model.predict(mm.transform(X[-150:]))
plt.plot(list(y['Adj. Close'][-151:]))
plt.plot(pred)
plt.show()